<a href="https://colab.research.google.com/github/SUB15LUIZ/Projetos_Luiz/blob/main/Consulta_SGS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install python-bcb

In [36]:
## Extração Informações Sistema Gerenciador de Séries Temporais - Bacen

import requests
import pandas as pd
import os
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

# Biblioteca Banco Central - sgs
# !pip install python-bcb
from bcb import sgs
#Conexão com o google drive
from google.colab import drive
drive.mount('/content/drive')


#Variaveis de caminho
caminho_entrada = "/content/drive/MyDrive/Colab Notebooks/Consulta_Bacen/Lista_de_series.xlsx"
saida_dir = "/content/drive/MyDrive/Colab Notebooks/Consulta_Bacen"

# caminho_entrada = "/content/drive/MyDrive/Colab Notebooks/Projeto_Consulta_SGS_api/Lista_de_series.xlsx"
# saida_dir = "/content/drive/MyDrive/Colab Notebooks/Projeto_Consulta_SGS_api"


def consultar_SGS(codigos_periodicidade: list, meses_retroativos=6, data_inicio=None):
    # Dataframe final extração
    df_base = None
    # DataFrame para registrar erros de consulta
    df_erros = pd.DataFrame(columns=['codigo', 'periodicidade', 'erro', 'detalhes'])

    if data_inicio is not None:
      dt_inicial = pd.to_datetime(data_inicio)
      dt_inicial = datetime(dt_inicial.year, dt_inicial.month, 1)
      # print (dt_inicial)
    else:
      data_atual = datetime.now()
      data_atual = datetime(data_atual.year, data_atual.month, 1)
      dt_inicial = data_atual - relativedelta(months=meses_retroativos)
      dt_inicial = dt_inicial.replace(day=1)

    # Entra item a item na lista
    for item in codigos_periodicidade:
        cod_item = item['codigo']
        periodicidade = item['periodicidade'].lower()


        try:
          dados = sgs.get(cod_item, start = dt_inicial)
          df_api = pd.DataFrame(dados)


          #Na primeira intereção ele so copia o dt_temp dps ele junta com que ja foi pesquisado
          if not df_api.empty:
            df_api = df_api.reset_index()
            df_api.rename(columns={'Date': 'data'}, inplace=True)


            if df_base is None:
              df_base = df_api
            else:
              df_base = pd.merge(df_base, df_api, on='data', how='outer')
          else:
            df_erros.loc[len(df_erros)] = {
                'codigo': cod_item,
                'periodicidade': periodicidade,
                'erro': 'EmptyData',
                'detalhes': 'Nenhum dado retornado'
        }

        except ValueError as e:
          df_erros.loc[len(df_erros)] = {
          'codigo': cod_item,
          'periodicidade': periodicidade,
          'erro': 'ValueError',
          'detalhes': str(e)[:100]
          }

        except Exception as e:
          df_erros.loc[len(df_erros)] = {
          'codigo': cod_item,
          'periodicidade': periodicidade,
          'erro': 'Exception',
          'detalhes': str(e)[:100]
          }

    return df_base,df_erros


# Ler o arquivo Excel
df_codigos = pd.read_excel(caminho_entrada)

# Cria DF para a extração dos casos Mensais
df_codigos_mensal = df_codigos[df_codigos['Frequencia_dado'] == 'Mensal']
df_codigos_mensal = df_codigos_mensal.drop_duplicates(subset='Codigo', keep='first')

# Cria DF para a extração dos casos Trimestrais
df_codigos_trimestral = df_codigos[df_codigos['Frequencia_dado'] == 'Trimestral']
df_codigos_trimestral = df_codigos_trimestral.drop_duplicates(subset='Codigo', keep='first')


# Transforma a DF em list para utilizar na consulta
codigos_periodicidade_mensal = df_codigos_mensal[['Codigo', 'Frequencia_dado']].rename(
    columns={"Codigo": "codigo", "Frequencia_dado": "periodicidade"}
).to_dict(orient='records')

codigos_periodicidade_trimestral = df_codigos_trimestral[['Codigo', 'Frequencia_dado']].rename(
    columns={"Codigo": "codigo", "Frequencia_dado": "periodicidade"}
).to_dict(orient='records')


#Chama a consulta e guarda em um dataframe
df_erros = None
saida_dados_mensal = None
saida_dados_trimestral = None


# saida_dados_mensal,df_erros_mensal = consultar_SGS(codigos_periodicidade_mensal)
# saida_dados_trimestral,df_erros_trimestral = consultar_SGS(codigos_periodicidade_trimestral)

saida_dados_mensal,df_erros_mensal = consultar_SGS(codigos_periodicidade_mensal, data_inicio ='2018-01-01')
saida_dados_trimestral,df_erros_trimestral = consultar_SGS(codigos_periodicidade_trimestral, data_inicio ='2018-01-01')


df_erros = pd.concat([df_erros_mensal, df_erros_trimestral], ignore_index=True)


# Verificando e salvando BKP e fazendo arquivos de dowloand - Mensais
if os.path.exists(os.path.join(saida_dir, "series_unificadas_mensal.csv")):

  bkp = pd.read_csv(os.path.join(saida_dir, "series_unificadas_mensal.csv"))
  bkp['data'] = pd.to_datetime(bkp['data'])

  bkp.to_csv(os.path.join(saida_dir, "series_unificadas_mensal_bkp.csv"), index=False, encoding='utf-8-sig')

  print ('Criado e salvo tabela bkp Mensal')

  # Verifica se consulta não foi nula
  if saida_dados_mensal is not None and not saida_dados_mensal.empty:

    saida_dados_mensal['data'] = pd.to_datetime(saida_dados_mensal['data'])

    # Concatena o Bkp com a Consulta Atual
    saida_dados_mensal = pd.concat([bkp, saida_dados_mensal], ignore_index=True)

    # Tira Duplicidades
    saida_dados_mensal = saida_dados_mensal.drop_duplicates(subset='data', keep='last')

    # Salva base a ser utilizada no processo
    saida_dados_mensal.to_csv(os.path.join(saida_dir, "series_unificadas_mensal.csv"), index=False, encoding='utf-8-sig')

    print("Arquivo Mensal Criado")

  else:
    print("Nenhum dado mensal retornado.")

else:
  # Verifica se consulta não foi nula
  if saida_dados_mensal is not None and not saida_dados_mensal.empty:
    saida_dados_mensal['data'] = pd.to_datetime(saida_dados_mensal['data'])

    # Salva base a ser utilizada no processo
    saida_dados_mensal.to_csv(os.path.join(saida_dir, "series_unificadas_mensal.csv"), index=False, encoding='utf-8-sig')
    print("Arquivo Mensal Criado")
  else:
    print("Nenhum dado mensal retornado no Mensal.")



# Verificando e salvando BKP e fazendo arquivos de dowloand - Trimestrais
if os.path.exists(os.path.join(saida_dir, "series_unificadas_trimestral.csv")):

  bkp = pd.read_csv(os.path.join(saida_dir, "series_unificadas_trimestral.csv"))
  bkp['data'] = pd.to_datetime(bkp['data'])

  bkp.to_csv(os.path.join(saida_dir, "series_unificadas_trimestral_bkp.csv"), index=False, encoding='utf-8-sig')

  print ('Criado e salvo tabela bkp Trimestral')

  # Verifica se consulta não foi nula
  if saida_dados_trimestral is not None and not saida_dados_trimestral.empty:

    saida_dados_trimestral['data'] = pd.to_datetime(saida_dados_trimestral['data'])
    # Concatena o Bkp com a Consulta Atual
    saida_dados_trimestral = pd.concat([bkp, saida_dados_trimestral], ignore_index=True)
    # Tira Duplicidades
    saida_dados_trimestral = saida_dados_trimestral.drop_duplicates(subset='data', keep='last')
    # Salva base a ser utilizada no processo
    saida_dados_trimestral.to_csv(os.path.join(saida_dir, "series_unificadas_trimestral.csv"), index=False, encoding='utf-8-sig')
    print("Arquivo Trimestral Criado")
  else:
    print("Nenhum dado mensal retornado no Trimestral.")

else:
  # Verifica se consulta não foi nula
  if saida_dados_mensal is not None and not saida_dados_mensal.empty:

    saida_dados_trimestral['data'] = pd.to_datetime(saida_dados_trimestral['data'])
    # Salva base a ser utilizada no processo
    saida_dados_mensal.to_csv(os.path.join(saida_dir, "series_unificadas_trimestral.csv"), index=False, encoding='utf-8-sig')
    print("Arquivo Trimestral Criado")
  else:
    print("Nenhum dado mensal retornado.")


print ("Consulta finalizada")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Criado e salvo tabela bkp Mensal
Arquivo Mensal Criado
Criado e salvo tabela bkp Trimestral
Arquivo Trimestral Criado
Consulta finalizada


In [ ]:
print (saida_dados_mensal)

In [37]:
print (saida_dados_trimestral)

         data  20718  20719  20720  20721  20722  20726   20727   22019  \
0  2024-11-01  21.66  16.45  35.44  13.34  18.59  51.58  346.65  268.49   
1  2024-12-01  21.73  17.20  34.89  14.34  19.68  50.69  330.95  167.21   
3  2025-02-01  23.81  18.22  38.91  15.57  24.23  54.42  340.18  220.91   
4  2025-03-01  24.57  18.85  38.36  16.47  23.68  54.81  349.21  186.64   
5  2025-04-01    NaN    NaN    NaN    NaN    NaN    NaN     NaN     NaN   
6  2025-05-01    NaN    NaN    NaN    NaN    NaN    NaN     NaN     NaN   
7  2018-01-01    NaN    NaN    NaN    NaN    NaN    NaN     NaN     NaN   
8  2018-04-01    NaN    NaN    NaN    NaN    NaN    NaN     NaN     NaN   
9  2018-07-01    NaN    NaN    NaN    NaN    NaN    NaN     NaN     NaN   
10 2018-10-01    NaN    NaN    NaN    NaN    NaN    NaN     NaN     NaN   
11 2019-01-01    NaN    NaN    NaN    NaN    NaN    NaN     NaN     NaN   
12 2019-04-01    NaN    NaN    NaN    NaN    NaN    NaN     NaN     NaN   
13 2019-07-01    NaN    N

In [ ]:
print (df_erros)